In [32]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import seaborn as sns
import pandas as pd
import cv2 
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
import os
from glob import glob
import shutil
import random
import time

In [33]:
"""input_folder_path_dog = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/dogs'
input_folder_path_cat = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats'
dog_contents = os.listdir(input_folder_path_dog)
cat_contents = os.listdir(input_folder_path_cat)

output_folder_path_dog = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/dogs_resized'
output_folder_path_cat = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats_resized'
cat_dog_output_train = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats_dogs_resized_train'
cat_dog_output_test = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/test_set/test_set/cats_dogs_resized_test'
target_size = (200, 200)
os.makedirs(output_folder_path_dog, exist_ok=True)
os.makedirs(output_folder_path_cat, exist_ok=True)
os.makedirs(cat_dog_output_train, exist_ok=True)
os.makedirs(cat_dog_output_test, exist_ok=True)

for dog_filename in dog_contents:
    img_path = os.path.join(input_folder_path_dog, dog_filename)
    try:
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, target_size)
        output_path = os.path.join(output_folder_path_dog, dog_filename)
        output_path_train = os.path.join(cat_dog_output_train, dog_filename)
        output_path_test = os.path.join(cat_dog_output_test, dog_filename)
        cv2.imwrite(output_path, img_resized)
        cv2.imwrite(output_path_train, img_resized)
        cv2.imwrite(output_path_test, img_resized)
        print(f"Resized: {output_path}")
    except Exception as e:
        print(f"Error processing {img_path}: {e}")

for cat_filename in cat_contents:
    img_path = os.path.join(input_folder_path_cat, cat_filename)
    try:
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, target_size)
        output_path = os.path.join(output_folder_path_cat, cat_filename)
        output_path_train = os.path.join(cat_dog_output_train, cat_filename)
        output_path_test = os.path.join(cat_dog_output_test, cat_filename)
        cv2.imwrite(output_path, img_resized)
        cv2.imwrite(output_path_train, img_resized)
        cv2.imwrite(output_path_test, img_resized)
        print(f"Resized: {output_path}")
    except Exception as e:
        print(f"Error processing {img_path}: {e}")"""

'input_folder_path_dog = \'/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/dogs\'\ninput_folder_path_cat = \'/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats\'\ndog_contents = os.listdir(input_folder_path_dog)\ncat_contents = os.listdir(input_folder_path_cat)\n\noutput_folder_path_dog = \'/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/dogs_resized\'\noutput_folder_path_cat = \'/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats_resized\'\ncat_dog_output_train = \'/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats_dogs_resized_train\'\ncat_dog_output_test = \'/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/test_set/test_set/cats_dogs_resized_test\'\ntarget_size = (200, 200)\nos.ma

In [43]:
def make_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer = 'he_uniform', padding = 'same', input_shape=(200,200, 3)))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(62, (3, 3), activation='relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(learning_rate = 0.001, momentum=0.9)
    model.compile(optimizer=opt, loss ='binary_crossentropy', metrics=['accuracy'])
    return model


In [44]:
def test_summary(history):
    plt.subplot(211)
    plt.title("Cross Entropy Loss")
    plt.plot(history.history['loss'], color='red', label='train')
    plt.plot(history.history['value_loss'], color='blue', label='test')
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='red', label='train')
    plt.plot(history.history['value_accuracy'], color='blue', label='test')
    filename= sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

In [46]:
def run_test():
    cat_dog_output_train = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/training_set/training_set/cats_dogs_resized_train'
    cat_dog_output_test = '/Users/kantasaito/Desktop/GitHub/MachineLearning/ImageClassification/archive/test_set/test_set/cats_dogs_resized_test'
    model = make_model()
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    training = datagen.flow_from_directory(cat_dog_output_train, class_mode='binary',batch_size=64, target_size=(200, 200))
    testing = datagen.flow_from_directory(cat_dog_output_test, class_mode='binary', batch_size=64, target_size=(200, 200))
    history = model.fit(training, steps_per_epoch=len(training), validation_data=testing, validation_steps=len(testing), epochs= 20, verbose=0)
    _, acc = model.evaluate(testing, steps=len(testing), verbose=0)
    print('> %.3f' % (acc * 100.0))
    test_summary(history)
run_test()
    

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0